In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchvision.datasets import ImageFolder
import os
from torchvision import models, transforms
import torch.optim as optim

In [20]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
batch_size = 128

In [22]:
#train and test data directory
train_dir = "train"
test_dir = "test"

In [23]:
# dataset has PILImage images of range [0, 1]. 
# We transform them to Tensors of normalized range [-1, 1]
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Compose([transforms.Resize((32,32))])])

In [24]:
#load the train and test data
dataset = ImageFolder(train_dir,transform=transform)
test_dataset = ImageFolder(test_dir,transform=transform)

In [25]:
#initiate data loaders
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)

In [26]:

dataloaders = [train_loader, test_loader]

In [27]:
classes = os.listdir("test")
classes

['21to30', '11to20', '61plus', '41to60', '0to10', '31to40']

In [28]:
model = models.resnet50(pretrained=True).to(device)

In [29]:
for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 6)).to(device)

# need to add an output of 6 classes

In [30]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

In [31]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in [0,1]:
            if phase == 0:
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 0:
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase])
            epoch_acc = running_corrects.double() / len(dataloaders[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model

In [32]:
model_trained = train_model(model, criterion, optimizer, num_epochs=10)

Epoch 1/10
----------
0 loss: 191.4038, acc: 50.9701
1 loss: 181.9496, acc: 52.8947
Epoch 2/10
----------
0 loss: 179.5140, acc: 55.7725
1 loss: 182.8616, acc: 52.5789
Epoch 3/10
----------
0 loss: 176.2909, acc: 56.7725
1 loss: 174.3109, acc: 54.7895
Epoch 4/10
----------
0 loss: 172.3434, acc: 58.8144
1 loss: 173.1639, acc: 55.8421
Epoch 5/10
----------
0 loss: 169.8688, acc: 59.3593
1 loss: 172.3107, acc: 56.6316
Epoch 6/10
----------
0 loss: 168.0936, acc: 59.9162
1 loss: 172.5330, acc: 56.7368
Epoch 7/10
----------
0 loss: 166.5691, acc: 60.4731
1 loss: 171.6458, acc: 55.4737
Epoch 8/10
----------
0 loss: 164.6494, acc: 61.1497
1 loss: 173.2237, acc: 57.5263
Epoch 9/10
----------
0 loss: 162.1891, acc: 62.0599
1 loss: 172.7193, acc: 57.4211
Epoch 10/10
----------
0 loss: 160.5947, acc: 62.7904
1 loss: 174.3746, acc: 55.6842


In [33]:
torch.save(model_trained.state_dict(), 'resnettuned.h5')

In [34]:
dummy_input = torch.randn(1, 3, 32, 32).to(device)
input_names = [ "actual_input" ]
output_names = [ "output" ]
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [35]:
# export as onnx
torch.onnx.export(model, 
                  dummy_input,
                  "restnetagepred.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names,
                  export_params=True,
                  )

### Save and Load Model

torch.save(model_trained.state_dict(), 'models/pytorch/weights.h5')

model = models.resnet50(pretrained=False).to(device)
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 6)).to(device)
model.load_state_dict(torch.load('models/pytorch/weights.h5'))